# **Setup**

In [1]:
!pip install datasets tensorflow

In [2]:
import pandas as pd
import numpy as np

from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from datasets import Dataset, DatasetDict
import cv2

import tensorflow as tf
from tensorflow.keras import layers, models

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
dataset = load_dataset("Marxulia/asl_sign_languages_alphabets_v03")

# **Data Exploration**

In [4]:
print(f"Total samples:", len(dataset['train']))
print(f"Categories: ", dataset['train'].features['label'].names)

Total samples: 10873
Categories:  ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [5]:
# Split the data into train and test sets (e.g., 80% train, 20% test)
train_images, test_images, train_labels, test_labels = train_test_split(
    dataset['train']['image'], dataset['train']['label'], test_size=0.2, random_state=42
)

# Create a DatasetDict to hold both the train and test sets
dataset = DatasetDict({
    'train':  Dataset.from_dict({'image': train_images, 'label': train_labels}),
    'test': Dataset.from_dict({'image': test_images, 'label': test_labels})
})

In [6]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 8698
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 2175
    })
})


# **Dataset**

In [7]:
def preprocess_image(example):
    image = example['image']
    image = tf.convert_to_tensor(np.array(image), dtype=tf.float32)
    image = tf.image.resize(image, [128, 128])
    image = image / 255.0
    label = example['label']
    return image, label

In [8]:
# Create a TensorFlow dataset using tf.data with preprocessing
train_dataset = tf.data.Dataset.from_generator(
    lambda: (preprocess_image(example) for example in dataset['train']),
    output_signature=(tf.TensorSpec(shape=(128, 128, 3), dtype=tf.float32), tf.TensorSpec(shape=(), dtype=tf.int64))
)


val_dataset = tf.data.Dataset.from_generator(
    lambda: (preprocess_image(example) for example in dataset['test']),
    output_signature=(tf.TensorSpec(shape=(128, 128, 3), dtype=tf.float32), tf.TensorSpec(shape=(), dtype=tf.int64))
)

In [9]:
train_dataset = train_dataset.batch(batch_size=64, drop_remainder=True).shuffle(5000).prefetch(tf.data.experimental.AUTOTUNE).repeat()
val_dataset = val_dataset.batch(batch_size=64, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE).repeat()

In [10]:
# Inspect a few batches of data
for image, label in train_dataset.take(2):  # Get the first batch
    print("Image shape:", image.shape)
    print("Label:", label)


Image shape: (32, 224, 224, 3)
Label: tf.Tensor(
[ 7 16  4  4  1 14 20 24 25 20 22  0  7  4  0 16  7  2 18 16  4 15  0 19
 14 21 23  6 21 25  7 24], shape=(32,), dtype=int64)
Image shape: (32, 224, 224, 3)
Label: tf.Tensor(
[ 7 16 24 15 12 22 14  0  3 24 10  8 17 23  3  9  3 10 17  3  6 20 25 17
 13 10 13 12  8  1  2 14], shape=(32,), dtype=int64)


In [11]:
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Input(shape=input_shape),

        # Convolutional Layer 1
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        # Convolutional Layer 2
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        # Convolutional Layer 3
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        # Flatten layer to convert 2D matrix to 1D vector
        layers.Flatten(),

        # Fully connected (Dense) layer
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),

        # Fully connected (Dense) layer
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),

        # Fully connected (Dense) layer
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.3),

        # Output layer (classification)
        layers.Dense(num_classes, activation='softmax')
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',  # For integer labels
                  metrics=['accuracy'])

    return model

# Create the CNN model
cnn_model = create_cnn_model(input_shape=(224, 224, 3), num_classes=26)

In [14]:
cnn_model.fit(
    train_dataset,
    epochs=40,
    steps_per_epoch=len(dataset['train']) // 64,
    validation_data=val_dataset,
    validation_steps=len(dataset['test']) // 64
)

Epoch 1/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 67s 112ms/step - accuracy: 0.0387 - loss: 3.3182 - val_accuracy: 0.0331 - val_loss: 3.2587
Epoch 2/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 45s 95ms/step - accuracy: 0.0364 - loss: 3.2584 - val_accuracy: 0.0364 - val_loss: 3.2591
Epoch 3/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 47s 94ms/step - accuracy: 0.0351 - loss: 3.2583 - val_accuracy: 0.0308 - val_loss: 3.2594
Epoch 4/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 46s 89ms/step - accuracy: 0.0380 - loss: 3.2579 - val_accuracy: 0.0340 - val_loss: 3.2590
Epoch 5/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 46s 85ms/step - accuracy: 0.0396 - loss: 3.2576 - val_accuracy: 0.0354 - val_loss: 3.2571
Epoch 6/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 47s 87ms/step - accuracy: 0.0463 - loss: 3.2528 - val_accuracy: 0.0499 - val_loss: 3.2482
Epoch 7/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 46s 87ms/step - accuracy: 0.0598 - loss: 3.2266 - val_accuracy: 0.0630 - val_loss: 3.2231
Epoch 8/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 55s 86ms/step - accuracy: 0.0755 - loss: 3.1759 -